In [3]:
import gym
import or_gym
from pyomo.environ import *
from or_gym.algos.math_prog_utils import *
import numpy as np

In [34]:
env_name = 'Knapsack-v1'
env = gym.make(env_name)
env.item_limits[-1] = 1
env.max_weight = 60

# Initialize model
m = ConcreteModel()

# Sets, parameters, and variables
m.W = env.max_weight
m.i = Set(initialize=env.item_numbers)
m.w = Param(m.i, 
    initialize={i: j for i, j in zip(env.item_numbers, env.item_weights)})
m.v = Param(m.i, 
    initialize={i: j for i, j in zip(env.item_numbers, env.item_values)})
m.b = Param(m.i,
    initialize={i: j for i, j in zip(env.item_numbers, env.item_limits)})
m.x = Var(m.i, within=NonNegativeIntegers)

@m.Constraint()
def weight_constraint(m):
    return sum(m.w[i] * m.x[i] for i in m.i) - m.W <= 0

@m.Constraint(m.i)
def item_constraint(m, i):
    return m.x[i] - m.b[i] <= 0

m.obj = Objective(expr=(
    sum([m.v[i] * m.x[i] for i in m.i])),
    sense=maximize)

In [35]:
m, results = solve_math_program(m, print_results=False)
m.obj.expr()

188.0

In [36]:
[m.x[i].value for i in m.x]

[0.0, 0.0, 0.0, 2.0, 3.0, 1.0]

In [33]:
env.item_limits

array([2, 1, 3, 4, 5, 6])